# test what should be load 
module avail
module load moebatch
module load MOE

# save the following script as a small shell script
#!/bin/bash
#SBATCH --job-name=moe_model
#SBATCH --output=moe_model.out
#SBATCH --time=01:00:00
#SBATCH --mem=8G

module load moebatch
moebatch moe_batch_modeling_repeats.svl

# in jupyter notebook
!sbatch run_moe_slurm.sh


In [ ]:
// === Setup ===
csv_path := "/path/to/antibody.csv";          // change to your CSV
output_base := "/path/to/moe_output/";        // change to your output base path
num_repeats := 3;

// === Load CSV ===
rows := io_ReadCSV[csv_path];

// === Loop Through Sequences ===
for i := 1 to length(rows) do
    row := rows[i];
    name := row["name"];
    vh := row["vh"];
    vl := row["vl"];
    seq := ["H" := vh, "L" := vl];

    for rep := 1 to num_repeats do
        tag := sprintf("%s_rep%d", name, rep);
        out_folder := output_base + tag;
        mkdir[out_folder];  // create folder

        printf("Running MOE model: %s (repeat %d)\n", name, rep);

        // Run homology model
        model := Antibody_Modeler[
            "seq_data" := seq,
            "num_models" := 1,
            "numbering_scheme" := "IMGT",
            "refine" := true
        ];

        // Save output
        pdb_path := sprintf("%s/%s.pdb", out_folder, tag);
        io_WritePDB[model, pdb_path];
    end for;
end for;
